<a href="https://colab.research.google.com/github/VijayaBhargavi198/5731Assignments/blob/master/In_class_exercise_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 10/29/2020)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [ ]:
!pip install polyglot 
!pip install pyicu
!pip install pycld2
!pip install Morfessor

In [ ]:
%%bash
polyglot download embeddings2.en pos2.en

In [21]:
import pandas as pd
import re
from polyglot.text import Text
df=pd.read_csv("/content/sample_data/Joker_user_reviews.csv")
preprocessed_data=[]
df["review"] = df["review"].str.replace(r"\W", " ").str.strip()# 1.To remove special characters and punctuations
df["review"]= df["review"].str.replace(r'\d+',"") #2.To remove Numbers
for a in df["review"]:   #3.To chane all the sentences to lower case
   a=a.lower()
   preprocessed_data.append(a)
df['Preprocessed data']=pd.DataFrame(preprocessed_data)
blob = df['Preprocessed data'].to_string()
text = Text(blob)
tags=text.pos_tags
#tags
sent_word = [x for (x,y) in tags if y not in ('NUM','PUNCT','ADP','AUX','CONJ','DET','INTJ','NOUN','PART','PRON','PROPN','SCONJ','SYM','X','VERB','ADV')] #took only adjectives beacuse they are the prime contributors for sentiment analysis
sent_word


['best',
 'de',
 'most',
 'early',
 'sad',
 'good',
 'recurring',
 'many',
 'intense',
 'more',
 'perfect',
 'amazing',
 'brilliant',
 'magnificent',
 'complete',
 'worthy',
 'best',
 'few',
 'unbelievable',
 'high',
 'incredible',
 'unconvincing',
 'best',
 'mental',
 'short',
 'cinematic',
 'good',
 'sure',
 'creative',
 'compelling',
 'new',
 'absolute',
 'silly',
 'best',
 'my',
 'first',
 'amazing',
 'enjoyable',
 'dark',
 'entertaining',
 'dark',
 'perfect',
 'best',
 'dark',
 'understandable',
 'interesting',
 'real',
 'brilliant',
 'hyped',
 'unique',
 'more',
 'outstanding',
 'strong',
 'mr',
 'stunning',
 'lite',
 'excellent',
 'amazing',
 'first',
 'unconvincing',
 'excellent',
 'good']

In [22]:
import collections
from operator import itemgetter
N = 200
frequency = collections.Counter(sent_word)
top_words = sorted(frequency.items(), key=itemgetter(1), reverse=True)[:N]
for i, (word, frequency) in enumerate(top_words, start=1):
    print("%d %s %d" % (i, word, frequency))


1 best 5
2 good 3
3 amazing 3
4 dark 3
5 more 2
6 perfect 2
7 brilliant 2
8 unconvincing 2
9 first 2
10 excellent 2
11 de 1
12 most 1
13 early 1
14 sad 1
15 recurring 1
16 many 1
17 intense 1
18 magnificent 1
19 complete 1
20 worthy 1
21 few 1
22 unbelievable 1
23 high 1
24 incredible 1
25 mental 1
26 short 1
27 cinematic 1
28 sure 1
29 creative 1
30 compelling 1
31 new 1
32 absolute 1
33 silly 1
34 my 1
35 enjoyable 1
36 entertaining 1
37 understandable 1
38 interesting 1
39 real 1
40 hyped 1
41 unique 1
42 outstanding 1
43 strong 1
44 mr 1
45 stunning 1
46 lite 1


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [23]:
# Sentiment Analysis using TextBlob
import nltk
nltk.download('punkt')
nltk.download('movie_reviews')
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
preprocessed_data=[]
df["review"] = df["review"].str.replace(r"\W", " ").str.strip()# 1.To remove special characters and punctuations
df["review"]= df["review"].str.replace(r'\d+',"") #2.To remove Numbers
for a in df["review"]:   #3.To chane all the sentences to lower case
   a=a.lower()
   preprocessed_data.append(a)
df['Preprocessed data']=pd.DataFrame(preprocessed_data)
text= df['Preprocessed data'].to_string()
blob = TextBlob(text, analyzer=NaiveBayesAnalyzer())
NaiveBayes_SentimentScore=blob.sentiment.classification
NaiveBayes_SentimentScore

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


'pos'

In [ ]:
# Sentiment Analysis using VADER 

In [ ]:
# Sentiment Analysis using TFIDF-based Support Vector Machine (SVM)